#### Attention is all you need

- 본 코드는 Attention is all you need : Transformer의 논문 내용과 일부 기타 자료를 참고하여 작성하였습니다.

- Tensorflow 대신 pytorch로 작성하였습니다.

- 모델에 대한 학습은 아직 진행해보지는 않았습니다. 추후 진행 예정 -> 변수 명에 오류가 있음

- 대신 Encoder, Decoder / Multi-head-attention / Residual connection(잔여 학습) + Normalization / Scaled-Dot-Product attention 구현은 하였습니다.

In [2]:
%%capture
!python -m spacy download en 
!python -m spacy download de 

In [3]:
import spacy 

2021-12-22 19:34:17.986223: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-22 19:34:17.986260: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
spacy_en = spacy.load("en_core_web_sm")
spacy_de = spacy.load("de_core_news_sm")

In [5]:
#간단히 토큰화 기능 써보기
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
    print("인덱스",i,':',token.text)

인덱스 0 : I
인덱스 1 : am
인덱스 2 : a
인덱스 3 : graduate
인덱스 4 : student
인덱스 5 : .


- 영어(English) 및 독일어(Deutsch) 토큰화 함수를 정의

In [6]:
# 독일어 문장을 토큰화하는 함수 
def tokenize_deutsch(text):
    return [token.text for token in spacy_de.tokenizer(text)]

In [7]:
# 영어 문장을 토큰화하는 함수
def tokenize_english(text):
    return [token.text for token in spacy_en.tokenizer(text)]

- 필드 라이브러리를 이용하여 데이터셋에 대한 구체적인 전처리 내용을 명시합니다.
- Seq2Seq 모델과는 다르게 batch_first 속성의 값을 True로 설정합니다.
- 번역 목표
  - 소스(src) : 독일어
  - 목표(trg) : 영어

In [8]:
from torchtext.data import Field,BucketIterator 

# Field : 어떻게 데이터를 전처리할 것인지를 설명 
# 보통 소문자로 
SRC = Field(tokenize=tokenize_deutsch, init_token="<sos>",eos_token="<eos>",lower=True,batch_first=True)
TRG = Field(tokenize=tokenize_english,init_token="<sos>",eos_token="<eos>",lower=True,batch_first=True)

- 대표적인 영어-독일어 번역 데이터셋인 Multi30k를 불러옵니다.

In [9]:
from torchtext.datasets import Multi30k 

train_dataset,valid_dataset,test_dataset = Multi30k.splits(exts=(".de",".en"),fields=(SRC,TRG))

In [10]:
print("학습 데이터셋(training dataset) 크기 :",len(train_dataset.examples))
print("평가 데이터셋(validation dataset) 크기 :",len(valid_dataset.examples))
print("검증 데이터셋(testing dataset) 크기 :",len(test_dataset.examples))

학습 데이터셋(training dataset) 크기 : 29000
평가 데이터셋(validation dataset) 크기 : 1014
검증 데이터셋(testing dataset) 크기 : 1000


- 한번 시험을 거쳐보도록 하겠습니다.
  - 훈련 데이터셋의 20번째 독일어 문장(src)가 주어지면, 
  - 이에 대한 훈련 데이터셋의 20번째 영어 문장(trg)의 쌍이 주어지게 됩니다.

In [11]:
print(vars(train_dataset.examples[20])['src'])
print(vars(train_dataset.examples[20])['trg'])


['ein', 'großes', 'bauwerk', 'ist', 'kaputt', 'gegangen', 'und', 'liegt', 'auf', 'einer', 'fahrbahn', '.']
['a', 'large', 'structure', 'has', 'broken', 'and', 'is', 'laying', 'in', 'a', 'roadway', '.']


- 필드(Field) : build_vocab을 이용, 영어 / 독일어 단어 사전을 생성

In [12]:
SRC.build_vocab(train_dataset,min_freq=2)
TRG.build_vocab(train_dataset,min_freq=2)

In [13]:
print('독일어 단어 차원 :',len(SRC.vocab))
print('영어 단어 차원 :',len(TRG.vocab))

독일어 단어 차원 : 7853
영어 단어 차원 : 5893


- TRG.vocab, 단어 군집 안에서 특정 문자를 입력하면 그 문자에 해당하는 index를 반환
- 이를 위해 stoi 내장 함수를 사용합니다.
- 만일 없는 단어라면, 0 / <sos> : 2 / <eos> : 3에 해당하고, 실제로 존재하는 어떤 단어의 경우 그에 대한 인덱스를 뱉어냅니다.

In [14]:
print(TRG.vocab.stoi["abcabc"]) #특정 string이 들어왔을 때 그에 대한 인덱스를 반환
print(TRG.vocab.stoi[TRG.pad_token])
print(TRG.vocab.stoi["<sos>"])
print(TRG.vocab.stoi["<eos>"])
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


- 한 문장에 포함된 단어가 순서대로 나열된 상태로 네트워크에 입력되어야 합니다.

  - 즉, 한 배치에 포함된 문장들이 가지는 단어의 개수에 큰 오차가 없도록 만들면 좋습니다.
     - 그러니까, 배치의 크기를 거의 균일하게 만든다고 보면 됩니다.
     - 다시 말해, 하나의 배치 안에 들어 있는 문장들의 Sequence 길이가 유사하도록 만들어서 
     - 가능한 한 길이가 짧은 문장의 padding token을 적게 들어가도록 만들어줍니다. 
     - 실제로 학습을 위해 네트워크에 들어가는 데이터의 차원을 최적화할 수 있습니다.


  - 이를 위해 BucketIlterator를 사용합니다.
  - Batch Size는 128로 임의로 잡도록 하겠습니다.

In [15]:
import torch 

#연구실 서버 상 제게 할당된 6번 서버에서 운용됩니다.
#각자의 상황에 맞게 :n을 조절하시면 될 듯 합니다.
device = torch.device('cuda:6' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128 

train_iterator,valid_iterator,test_iterator = BucketIterator.splits( 
    (train_dataset,valid_dataset,test_dataset), 
    batch_size = BATCH_SIZE,
    device=device
)



- 각 Batch마다 확인하므로 시간복잡도가 n^2이 됩니다.
- 따라서 여건 상 첫 번째 Batch만 확인하도록 하겠습니다.
- (128,28) shape의 배치가 만들어 집니다.

In [16]:
for i, batch in enumerate(train_iterator):
    ssrc = batch.src
    ttrg = batch.trg 

    print("첫 번째 배치 크기 :",ssrc.shape)

    for i in range(ssrc.shape[1]):
        print("인덱스 ",i,":", ssrc[0][i].item())

    break 


첫 번째 배치 크기 : torch.Size([128, 28])
인덱스  0 : 2
인덱스  1 : 5
인덱스  2 : 25
인덱스  3 : 7
인덱스  4 : 6
인덱스  5 : 102
인덱스  6 : 882
인덱스  7 : 297
인덱스  8 : 12
인덱스  9 : 14
인덱스  10 : 136
인덱스  11 : 11
인덱스  12 : 6
인덱스  13 : 2255
인덱스  14 : 20
인덱스  15 : 86
인덱스  16 : 4
인덱스  17 : 3
인덱스  18 : 1
인덱스  19 : 1
인덱스  20 : 1
인덱스  21 : 1
인덱스  22 : 1
인덱스  23 : 1
인덱스  24 : 1
인덱스  25 : 1
인덱스  26 : 1
인덱스  27 : 1


#### 하나의 Batch를 확인했더니..
- 1 : 의미가 없는 padding token에 해당합니다.
- 2 : <sos>에 해당합니다.
- 3 : <eos>에 해당합니다.

- 즉 이 문장은 총 길이가 18인 하나의 문장을 뜻합니다.

#### Transformer : RNN, CNN을 쓰지 않고, Attention(self-attention / Multi-head-attention / Scaled-dot product attention)으로 이루어집니다.

- 이를 논문의 내용과 유사하게 구현해보도록 하겠습니다.



- Attention을 이루는 세 가지 구성 요소
  - Query : 하나의 Sequence에서 나와 가장 관련이 있는 단어가 무엇인지 묻는 주체(Subject)
  - Key : 해당 Query가 바라보는 대상(본인 포함. 그래서 self-attention)
  - Value : 소맥을 통과한 이후 가중값 계산을 위해 곱해지는 W




- Hyperparameter
  - hidden_dim : 하나의 단어에 대한 임베딩 차원. 즉 len(words[0])
  - n_heads : head의 개수. multi-head attention에서 decoder에서 만들어 내는 head의 개수를 말합니다. 많아지면 많아질 수록 더욱 세밀한 연산이 가능합니다. 
    - 다른 표현으로 head의 개수는 scaled dot-product attention의 개수를 말하기도 합니다.
  - dropout_ratio : Fully-connected layer를 통과할 때 임의의 한 Layer의 노드 집합 가운데에서 전체 노드 대비 비활성화되는 노드의 비율  

- 임베딩 차원을 head의 개수로 나눠주게 되면 각각 Q,K,V의 가로 차원이 나오게 됩니다.
- 세로 길이는 임베딩 차원이 됩니다.

In [17]:
import torch.nn as nn 

class Multi_Head_Attention_Layer(nn.Module):

    #생성자 결정하고 각각의 parameter 순서를 정해줍니다. 

    def __init__(self, hidden_dim,n_heads,dropout_ratio,device):
        super().__init()
        #예외 처리
        assert hidden_dim % n_heads ==0


        self.hidden_dim = hidden_dim
        #임베딩 차원. 즉 한 문장에 대해 쭉 늘어뜨리는 정도 
        self.n_heads = n_heads 
        #헤드의 개수. 임베딩 차원을 헤드의 개수로 나누면 Q,K,V의 차원이 나오게 된다.
        self.head_dim = hidden_dim//n_heads 
        #즉 각 헤드에서의 임베딩 차원이 됩니다. 

        self.FC_Q = nn.Linear(hidden_dim,hidden_dim)
        self.FC_K = nn.Linear(hidden_dim,hidden_dim)
        self.FC_V = nn.Linear(hidden_dim,hidden_dim)
        #Q,K,V에 적용될 각각의 레이어의 차원을 넣어줍니다. 

        self.FC_O = nn.Linear(hidden_dim,hidden_dim)
        #마지막에 elementwise하게 연산될 것.

        self.drop_out = nn.Dropout(dropout_ratio)
        #하이퍼 파라미터

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
    


    #FFW 
    def forward(self,q,k,v,mask=None):
        batch_size = q.shape[0]

        #원래 각각의 Q,K,V는 2차원이었으나, Batch 크기만큼 들어오게 되므로
        #일괄적으로 Batch size 만큼 차원이 늘어나게 될 것

        #q.shape : (batch_size,len(q),hidden_dim)
        #k.shape : (batch_size,len(k),hidden_dim)
        #v.shape : (batch_size,len(v),hidden_dim)

        WQ = self.FC_Q(q)
        WK = self.FC_K(k)
        WV = self.FC_V(v)

        #각 헤드에 대한 연산을 위하여
        #기존의 hidden_dim을 n_heads * head_dims 로 변경 쉽게 말하면 쪼개준다
        #n_heads(h) 개의 서로 다른 attention만큼 학습을 진행합니다. 

        WQ = WQ.view(batch_size,-1,self.n_heads,self.head_dim).permute(0,2,1,3)
        WK = WK.view(batch_size,-1,self.n_heads,self.head_dim).permute(0,2,1,3)
        WV = WV.view(batch_size,-1,self.n_heads,self.head_dim).permute(0,2,1,3)
        #head의 개수만큼으로 나눠서 벡터들을 생성하고 길이는 head의 차원으로 설정합니다. 

        # Q.shape : [batch_size,n_heads,len(q),head_dim]
        # K.shape : [batch_size,n_heads,len(k),head_dim]
        # V.shape : [batch_size,n_heads,len(v),head_dim]

        # Attention Energy 계산

        each_energy = torch.matmul(WQ,WK.permute(0,1,3,2)) / self.scale 

        #에너지도 동일한 차원으로 맞춰준다. 
        #이제 이에 대한 소맥을 취한다. 

        if mask is not None:
            #mask 처리된 부분이 소맥에 거의 반영되지 않도록 
            each_energy = each_energy.masked_fill(mask==0,-1e10)
        

        #이제 어텐션 스코어를 각각 단어에 대해 계산합니다. 
        attention = torch.softmax(each_energy,dim=-1)

        #attenion의 차원 : [batch_size,n_heads,len(q),len(k)]

        #현재 Scaled-dot product Attention을 계산하는 중임 그 벤다이어그램을 생각하셈
        res = torch.matmul(self.dropout(attention),WV)

        res = res.permute(0,2,1,3).contiguous()

        #한꺼번에 쭉 늘어뜨려 Concat의 효과를 유도
        res = res.view(batch_size,-1,self.hidden_dim)
        
        res = self.FC_O(res)

        return res,attention


Position-wise Feedforward Architecture


- 입력.shape와 출력.shape는 동일합니다.

- 하이퍼 파라미터
  - hidden_dim : 하나의 단어의 임베딩 길이(차원)
  - PFRD_dim : FFW Layer에서의 임베딩 길이(차원)
  - dropout_ratio : 드롭아웃의 비율 

In [18]:
class Position_Wise_Feed_Forward_Layer(nn.Module):
    def __init__(self,hidden_dim,PFRD_dim,dropout_ratio):
        super().__init__() 

        self.FC_1 = nn.Linear(hidden_dim,PFRD_dim)
        self.FC_2 = nn.Linear(PFRD_dim,hidden_dim)

        self.dropout = nn.Dropout(dropout_ratio)
    
    def Forward(self,res):
        # 전부 선형대수연산임.
        # res의 shape : [batch_size,len(seq),hidden_dim]

        res = self.dropout(torch.relu(self.FC_1(res)))

        # res의 shape : [batch_size,len(seq),PFRD_dim]

        res = self.FC_2(res)

        # res의 shape : [batch_size,len(seq),hidden_dim]

        return res

- 앞서 정의한 Attention과 FFN을 사용하여, Encoder / Decoder를 정의하도록 하겠습니다.

Encoder 

- 낱개 Encoder를 정의
  - premise : 입력.shape와 출력.shape가 동일
  - 같은 구조의 Encoder를 중첩하여 사용합니다. 



- 총 4개의 Layer로 이루어져 있습니다.
  - Multi-head Attention, Residual, FeedForward Layer, Residual
  - Residual 잔여 연산은 LayerNorm(x+Sublayer(x)) 을 의미합니다.

- Hyperparameter
  - hidden_dim : 하나의 단어에 대한 임베딩 차원
  - n_heads : head의 개수 == scaled-dot-product attention의 개수
  - PFRD_dim : FFL에서의 임베딩 차원 
  - dropout_ratio : 드롭아웃의 비율 

- <pad> 토큰을 반영하지 않도록 마스크 값을 0으로 설정

In [19]:
class EncoderLayer(nn.Module):
    def __init__(self,hidden_dim,n_heads,PFRD_dim,dropout_ratio,device):
        super().__init__() 
    
        self.self_attention_layer_norm = nn.LayerNorm(hidden_dim)

        self.Feed_Forward_Layer_Norm = nn.LayerNorm(hidden_dim)

        self.Self_Attention = Multi_Head_Attention_Layer(hidden_dim,n_heads,dropout_ratio,device)

        self.Positionwise_Feedforward = Position_Wise_Feed_Forward_Layer(hidden_dim,PFRD_dim,dropout_ratio) 

        self.dropout = nn.Dropout(dropout_ratio)


    #하나의 임베딩이 복제되어 Q,K,V로 입력됨 
    # 왜 복제를 하느냐하면 Residual 연산을 하기 위함. LayerNorm(x+Sublayer(x))

    def Forward(self,src,src_mask):

        #src : 우리는 지금 독일어 -> 영어 이므로 
        #src mask : [batch_size,len(src)] 
        # 

        ################### Self Attention #################

        #왜 src,src,src 이냐하면 Q,K,V만큼 복제를 해야 하기 때문
        ssrc, tmp = self.Self_Attention(src,src,src,src_mask)    

        #dropout, residual connection and Layer norm

        src = self.self_attention_layer_norm(src+self.dropout(ssrc))

        # src : [batch_size, len(src), hidden_dim]

        #positionalwiseFFW
        ssrc = self.Positionwise_Feedforward(src)

        #dropout,residual and layer norm

        src = self.Feed_Forward_Layer_Norm(src+self.dropout(ssrc))

        #그냥 논문의 내용을 충실하게 구현중임.

        return src 

인코더(Encoder) 전체를 구현합니다.

- 논문에서 나온 대로 N개 만큼의 인코더를 구현합니다.

- 하이퍼파라미터
  - input_dim : 하나의 단어에 대한 OHE 길이(차원)
  - hidden_dim : 하나의 단어에 대한 임베딩 길이(차원)
  - n_layers : 논문에서 나온 N을 의미, 즉 인코딩 레이어의 개수
  - n_heads : Multi-head attention을 위한 head의 개수. head의 개수는 Scaled-dot-product attention의 개수과 같다.
  - PFRD_dim : FFW 레이어에서의 임베딩 길이(차원)
  - dropout_ratio : 드롭아웃 비율
  - max_length : 문장 내 최대 단어의 개수

In [22]:
class Encoder(nn.Module):

    def __init__(self,input_dim,hidden_dim,n_layers,n_heads,PFRD_dim,dropout_ratio,device,max_length=100):
        super().__init__() 

        self.device = device 

        self.token_embedding = nn.Embedding(input_dim,hidden_dim)
        #처음 인풋에 대한 OHE와, Positional Embeding을 위한 hidden_dim까지 

        self.pos_embedding = nn.Embedding(max_length,hidden_dim)

        self.Layers = nn.ModuleList([EncoderLayer(hidden_dim,n_heads,PFRD_dim,dropout_ratio,device) for i in range(n_layers)])
        # N개의 Layer만큼 추가

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def Forward(self,src,src_mask):

        # src: [batch_size, src_len]
        # src_mask: [batch_size, src_len]

        batch_size = src.shape[0]
        src_len = src.shape[1]

        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

        # pos: [batch_size, src_len]

        # 소스 문장의 임베딩과 위치 임베딩을 더한 것을 사용
        src = self.dropout((self.token_embedding(src) * self.scale) + self.pos_embedding(pos))

        # src: [batch_size, src_len, hidden_dim]

        # 모든 인코더 레이어를 차례대로 거치면서 순전파(forward) 수행
        for layer in self.Layers:
            src = layer(src, src_mask)

        # src: [batch_size, src_len, hidden_dim]

        return src # 마지막 레이어의 출력을 반환

        

Decoder

- 낱개의 디코더 레이어
  - premise : 입력.shape == 출력.shape 
  - Encoder와 마찬가지로 동일한 n_layers만큼 중첩하여 사용합니다. 
  - Decoder에서는 Encoder의 ouput을 k,v로 사용할 수 있도록, Q 역할을 할 수 있는 또 다른 Multi-head-Attention layer가 있다.
  - Multi-head-Attention 2개 / Residual + Normalization 2개 / Feed Forward 1개 / Residual + Normalization 1개 

- 하이퍼 파라미터
  - Encoder와 동일

- 타겟 문장에서 각 단어들은 다음 단어를 미리 알 수 없도록 만들기 위해 마스크를 사용합니다. 
  - 왜냐하면, Encoder에서 나온 문장 전체를 참고하게 되면 객관적인 모델 평가에 있어 악영향을 줄 수도 있기 때문입니다. 
  

In [27]:
class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        self.self_attention_layer_norm = nn.LayerNorm(hidden_dim)
        self.encoding_attention_layer_norm = nn.LayerNorm(hidden_dim)
        self.Feed_Forward_Layer_Norm = nn.LayerNorm(hidden_dim)
        self.Self_Attention = Multi_Head_Attention_Layer(hidden_dim, n_heads, dropout_ratio, device)
        self.Encoder_Attention = Multi_Head_Attention_Layer(hidden_dim, n_heads, dropout_ratio, device)
        self.Positionwise_Feedforward = Position_Wise_Feed_Forward_Layer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더의 출력 값(enc_src)을 어텐션(attention)하는 구조
    def forward(self, trg, enc_src, trg_mask, src_mask):

        # trg: [batch_size, trg_len, hidden_dim]
        # enc_src: [batch_size, src_len, hidden_dim]
        # trg_mask: [batch_size, trg_len]
        # src_mask: [batch_size, src_len]

        # self attention
        # 자기 자신에 대하여 어텐션(attention)
        _trg, _ = self.Self_Attention(trg, trg, trg, trg_mask)

        # dropout, residual connection and layer norm
        trg = self.self_attention_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]

        # encoder attention
        # 디코더의 쿼리(Query)를 이용해 인코더를 어텐션(attention)
        _trg, attention = self.Encoder_Attention(trg, enc_src, enc_src, src_mask)

        # dropout, residual connection and layer norm
        trg = self.encoding_attention_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]

        # positionwise feedforward
        _trg = self.Positionwise_Feedforward(trg)

        # dropout, residual and layer norm
        trg = self.Feed_Forward_Layer_Norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return trg, attention

디코더 전체

- 하이퍼 파라미터
  - output_dim : 하나의 단어에 대한 OHE 길이(차원)
  - hidden_dim : 하나의 단어에 대한 임베딩 길이(차원)
  - n_layers : 인코더의 개수 == 디코더의 개수
  - n_heads : head의 개수 == Scaled Dot Product attention의 개수 
  - PFRD_dim : FFW Layer에서의 임베딩 길이(차원)
  - dropout_ratio : 드롭아웃 비율 
  - max_length : 문장 내 최대 단어의 개수 

- 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없게 하였습니다..
  - 임의의 t 위치에 대한 단어 output은 1부터 t-1까지만 확인할 수 있도록 처리합니다.

In [25]:
class Decoder(nn.Module):
    def __init__(self,output_dim,hidden_dim,n_layers,n_heads,PFRD_dim,dropout_ratio,device,max_length=100):
        super().__init__()

        self.device = device

        self.token_embedding = nn.Embedding(output_dim,hidden_dim)

        self.pos_embedding = nn.Embedding(max_length,hidden_dim)

        self.Layers = nn.ModuleList([DecoderLayer(hidden_dim,n_heads,PFRD_dim,dropout_ratio,device) for i in range(n_layers)])

        self.FC_OUT = nn.Linear(hidden_dim,output_dim)

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    
    def Forward(self,trg,enc_src,trg_mask,src_mask):

        batch_size = trg.shape[0]
        trg_len=trg.shape[1]

        pos = torch.arange(0,trg_len).unsqueeze(0).repeat(batch_size,1).to(self.device)

        trg = self.dropout((self.token_embedding(trg) * self.scale) + self.pos_embedding(pos))

        # trg: [batch_size, trg_len, hidden_dim]

        for layer in self.Layers:
            # 소스 마스크와 타겟 마스크 모두 사용
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)

        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        output = self.FC_OUT(trg)

        # output: [batch_size, trg_len, output_dim]

        return output, attention

Transformer

- 전체 트랜스포머의 모델을 정의합니다.
- 입력이 들어왔을 때 앞서 정의한 인코더와 디코더를 거쳐 출력 문장을 만들어냅니다.

In [26]:
class Transformer(nn.Module):
    def __init__(self, Encoder, Decoder, src_pad_idx, trg_pad_idx, device):
        super().__init__()

        self.Encoder = Encoder
        self.Decoder = Decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    # 소스 문장의 <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정
    def make_src_mask(self, src):

        # src: [batch_size, src_len]

        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        # src_mask: [batch_size, 1, 1, src_len]

        return src_mask

    # 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용
    def make_trg_mask(self, trg):

        # trg: [batch_size, trg_len]

       
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)

        # trg_pad_mask: [batch_size, 1, 1, trg_len]

        trg_len = trg.shape[1]

        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()

        # trg_sub_mask: [trg_len, trg_len]

        trg_mask = trg_pad_mask & trg_sub_mask

        # trg_mask: [batch_size, 1, trg_len, trg_len]

        return trg_mask

    def forward(self, src, trg):

        # src: [batch_size, src_len]
        # trg: [batch_size, trg_len]

        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        # src_mask: [batch_size, 1, 1, src_len]
        # trg_mask: [batch_size, 1, trg_len, trg_len]

        enc_src = self.Encoder(src, src_mask)

        # enc_src: [batch_size, src_len, hidden_dim]

        output, attention = self.Decoder(trg, enc_src, trg_mask, src_mask)

        # output: [batch_size, trg_len, output_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return output, attention